In [1]:
from KNN import KNN_LOO
from utils import read_data, standardize, distance_all_datapoints
import time
import distance_calc
import matplotlib.pyplot as plt

### Parameter choices for KNN-LOO

In [2]:
distance_method = distance_calc.euclidean_squared
# distance_method = distance_calc.manhattan

verbose = True  # For more print statements.
verbose = False  # For less print statements.

k_list = range(1, 99)  # n_of_datapoints-1, -1 for the test datapoint

# filename = 'QSAR_reduced_40.csv'   # nelem=3200  (n=100, p=32)
# filename = 'QSAR_3_large_data.csv'   # nelem=41.6k  (n=1300, p=32)
filename = 'QSAR_2.csv'   # nelem=41.6k  (n=1300, p=32)

### Read data

In [3]:
print('Reading data...', end=' ')
unstandardized_data = read_data(filename)  # List[lists], no headers
print('Done!')

# For later use:
n_of_datapoints = len(unstandardized_data)
n_of_features = len(unstandardized_data[0]) - 2  # Exclude name and class.
nelem = n_of_datapoints * n_of_features

Reading data... Done!


### Standardize data

In [12]:
print('Standardizing data...', end=' ')

# Standardize the values of all datapoints (these do not include the first two columns, as they contain the name and class):
standardized_values = standardize([i[2:] for i in unstandardized_data],method='custom')
print('Done!')

Standardizing data... Done!


### Create list of tuples

In [14]:
# For each datapoint, add the name, class and standardized values together into a tuple, 
#   and store these tuples in the list 'data':
data = []
for index in range(len(unstandardized_data)):
    datapoint = [unstandardized_data[index][0],unstandardized_data[index][1]]
    for val in standardized_values[index]:
        datapoint.append(val)
    data.append(tuple(datapoint))


### Calculate distances

In [ ]:
print('Calculating distances... (this will take approx. '
      f'{4.65/10000 * nelem:.0f} s)', end=' ')
start_time = time.time()
# Convert list of lists -> list of tuples
data_list_of_tuples = [tuple(internal_list) for internal_list in data]

# Calculate distances between all datapoints
distance_dict = distance_all_datapoints(data_list_of_tuples, distance_method)
end_time = time.time()
computation_time_dists = end_time - start_time
print('Done!')

## QUESTION 2
----------
Determine for at least 5 different values of k what the error score
of your k-nearest neighbour algorithm is, i.e., the number of times 
that the algorithm generates a different label than the real label 
when run for all 100 data points with the leave-one-out procedure.


In [16]:
##%% for one run

# test_datapoint = data[0]
# training_data = data[1:]

# print(f'''{KNearestNeighborsLOO(
#                 training_data, test_datapoint, k=50,
#                 distance_method = euclidean_squared,
#                 verbose=True).outcome = }''')


##%% Running the algorithm.
error_score_dict = {}
error_score_dict_for_plot = {}
computation_time_dict = {}
# NNs_dist_dict_per_k = {}
# lowest_dist_keys_per_k = {}
for k in k_list:
    print(f'k = {k}\n')
    
    start_time = time.time()
    
    KNN_outcome_list = []  # To keep track of the outcomes during the loop
    
    # Run for every single datapoint as test datapoint
    for idx, datapoint_i in enumerate(data_list_of_tuples):
        distances_test_to_training = distance_dict[datapoint_i]
        n_of_digits = len(str(n_of_datapoints))
        print(f'-- Datapoint {idx+1:0{n_of_digits}d} of {n_of_datapoints} (k={k}) --')
        
        # Run KNN and save the outcome: True if correct
        KNN_outcome_list.append(
            KNN_LOO(distances_test_to_training, k, verbose).outcome
            )
        
        # ## Testjes
        # verbose=False
        # NNs_dist_dict_per_k[k] = (KNN_LOO(
        #         distances_test_to_training, k, verbose)).NNs_dist_dict
        # lowest_dist_keys_per_k[k] = (KNN_LOO(
        #         distances_test_to_training, k, verbose))._lowest_dist_keys
        # ##/Testjes
    # Computation time check
    end_time = time.time()
    computation_time = end_time - start_time
    print('\n')
    print('==================================================================')
    print(f'For {n_of_datapoints} datapoints with {len(datapoint_i)} '
          f'features, k={k}, with distance method = {distance_method}, '
          f'it takes {computation_time:.3f} s to run the KNN algorithm and '
          'compute the error scores.')
    # Determine error score i.e. number of times that the algorithm
    # generates a different label than the real label, and save it 
    # for the current *k*.
    error_score = KNN_outcome_list.count(False)  # int: 0-100
    error_score_proportion = str(error_score) + '/' + str(n_of_datapoints)
    error_score_dict[k] = error_score_proportion
    error_score_dict_for_plot[k] = error_score
    computation_time_dict[k] = computation_time
    print('==================================================================')
    print('Error scores per k:')
    print(error_score_dict)    
    print('==================================================================')
    print('Computation time per k:')
    print(computation_time_dict)

# Plot: k vs error score
x = list(error_score_dict_for_plot.keys())
y = list(error_score_dict_for_plot.values())
plt.scatter(x, y)
plt.plot(x, y, '--')
plt.title(f'Error score as a function of k (n={n_of_datapoints})')
plt.xlabel('k')
plt.ylabel(f'Error score (n = {n_of_datapoints})')
plt.show()

# Plot: k vs computation time
y = list(computation_time_dict.values())
plt.scatter(x, y)
plt.plot(x, y, '--')
plt.suptitle(f'Computation time as a function of *k* (# elem={nelem})')
plt.title(f'(computing distances took {computation_time_dists:.2f} s)')
plt.xlabel('k')
plt.ylabel('Computation time [s]')
plt.show()

print('==================================================================')
print(f'Calculate dists time: {computation_time_dists:.02f} s')
print(f'Calculate dists time per datapoint: {computation_time_dists/n_of_datapoints:.05f} s')

k = 1

-- Datapoint 0001 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 0002 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 0003 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... T

Evaluating performance of KNN... The class of the test datapoint has NOT been correctly predicted: 
predicted class = ppar 
actual class = cox2.

-- Datapoint 0170 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'ppar': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: ppar
Evaluating performance of KNN... The class of the test datapoint has NOT been correctly predicted: 
predicted class = ppar 
actual class = cox2.

-- Datapoint 0171 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'ppar': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: ppar
Evaluating performance of KNN... The class of the test datapoint has NOT been correctly predicted: 
predicted class = ppar 
actual class = cox2.

-- Datapoint 0172 of 1300 (k=1) --
Determining the *k* close

Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 0289 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'ppar': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: ppar
Evaluating performance of KNN... The class of the test datapoint has NOT been correctly predicted: 
predicted class = ppar 
actual class = cox2.

-- Datapoint 0290 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'ppar': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: ppar
Evaluating performance of KNN... The class of the test datapoint has NOT been correctly predicted: 
predicted class = ppar 
actual class = cox2.

-- Datapoint 0291 of 1300 (k=1) --
Determining the *k* closest (training) datapoi

Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 0475 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 0476 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 0477 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2':

Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 0615 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 0616 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 0617 of 1300 (k=1) --
Determining the *k* closest (training) datapoints to the test datapoint...

...

Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 1184 of 1300 (k=6) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2': 6}
Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 1185 of 1300 (k=6) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2': 6}
Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 1186 of 1300 (k=6) --
Determining the *k* closest (training) datapoints to the test datapoint...

-- Datapoint 0038 of 1300 (k=7) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2': 7}
Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 0039 of 1300 (k=7) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2': 5, 'thrombin': 1, 'ppar': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Evaluating performance of KNN... The class of the test datapoint has been correctly predicted (cox2).

-- Datapoint 0040 of 1300 (k=7) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'cox2': 6, 'ppar': 1}
Determining the most occurent classes of the NNs... Done!
Predicted class: cox2
Eval

Evaluating performance of KNN... The class of the test datapoint has NOT been correctly predicted: 
predicted class = ppar 
actual class = cox2.

-- Datapoint 0163 of 1300 (k=7) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'ppar': 7}
Determining the most occurent classes of the NNs... Done!
Predicted class: ppar
Evaluating performance of KNN... The class of the test datapoint has NOT been correctly predicted: 
predicted class = ppar 
actual class = cox2.

-- Datapoint 0164 of 1300 (k=7) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'ppar': 7}
Determining the most occurent classes of the NNs... Done!
Predicted class: ppar
Evaluating performance of KNN... The class of the test datapoint has NOT been correctly predicted: 
predicted class = ppar 
actual class = cox2.

-- Datapoint 0165 of 1300 (k=7) --
Determining the *k* close

-- Datapoint 0339 of 1300 (k=7) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'thrombin': 7}
Determining the most occurent classes of the NNs... Done!
Predicted class: thrombin
Evaluating performance of KNN... The class of the test datapoint has NOT been correctly predicted: 
predicted class = thrombin 
actual class = cox2.

-- Datapoint 0340 of 1300 (k=7) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'thrombin': 7}
Determining the most occurent classes of the NNs... Done!
Predicted class: thrombin
Evaluating performance of KNN... The class of the test datapoint has NOT been correctly predicted: 
predicted class = thrombin 
actual class = cox2.

-- Datapoint 0341 of 1300 (k=7) --
Determining the *k* closest (training) datapoints to the test datapoint... Done!
Occurences of the classes in the NNs are:  {'thrombin': 7}
Determi

KeyboardInterrupt: 